# Simulation cut study

In [1]:
import numpy as np
import pandas as pd
import uproot
import boost_histogram as bh
import functools as ft
import matplotlib.pyplot as plt

%matplotlib widget

In [2]:
plt.style.use('seaborn')
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.formatter.limits'] = (-3,3)
plt.rcParams['axes.formatter.use_mathtext'] = True
# plt.rcParams['font.size']= 16
# plt.rcParams['axes.labelsize'] = 14
# plt.rcParams['axes.titlesize'] = 14
# plt.rcParams['xtick.labelsize'] = 10
# plt.rcParams['ytick.labelsize'] = 10

In [3]:
#MCpart_root_file = "/home/pauln/code/pi0chanser/output/chanser_MC_12k-15k/pauln/Pi0_PID_photcomb/ParticleVariables_0.root"

tree= "FINALOUTTREE"

# data_root_file = "/w/work3/home/pauln/analysis/processed/chanser_ANAsub/pauln/Pi0_PID_fullcomb/FinalState.root"
# data = uproot.open(data_root_file)[tree]
# data_df = data.pandas.df()

#MC_root_file = "/w/work3/home/pauln/analysis/processed/chanser_MCsub/pauln/Pi0_PID_fullcomb/FinalState.root"
MC_root_file = "/home/pauln/code/pi0chanser/output/chanser_MC15_gammatruth/pauln/Pi0_PID_exclcomb/FinalState.root"

data = uproot.open(MC_root_file)[tree]
MC_df = data.pandas.df()

# data = uproot.open(MCpart_root_file)['particle']
# MCpart_df = data.pandas.df()

In [4]:
def hist1D(ax, df, var, bins = None, cuts=None, y_err=False, color=None, lalpha=1, falpha=0.4, normed=False, filled = False, density = False, params={}, label=None):
    #select next color in colormap for use in line and fill.
    if color is None:
        color = next(ax._get_lines.prop_cycler)['color']
    
    if cuts is not None: 
        data = df[var[0]][cuts]
    else:
        data = df[var[0]]
        
    #calculate area under the curve if wanting normalised plot    
    weights = None
    if normed is True:
        entries, edges = np.histogram(
            data,
            range = (var[1][1], var[1][2]),
            bins = var[1][0]        
        )
      
        integral = sum(np.diff(edges)*entries)
        weights = np.ones_like(data)/integral
        
    if bins is None: 
        bins = var[1][0]
    
    #Draw histogram
    entries,edges,_ = ax.hist(data,
    range = (var[1][1], var[1][2]),
    bins = bins,
    density = density,
    weights=weights,
    histtype = 'step',
    linewidth = 1.5,
    alpha=lalpha,
    color = color,
    label=label,
    **params
    )
 
    #fill area under histo if set
    if filled is True:
        # brute-force: drawing it again to allow separate alpha(opacity) to be used
        h = ax.hist(data,
            range = (var[1][1], var[1][2]),
            bins = bins,
            density = density,
            weights=weights,
            histtype = 'step',
            linewidth = None,
            fill = True,
            alpha = falpha,
            color = color
            )
        
    #error bars    
    if y_err is True:
        if normed is True: 
            yerr = np.sqrt(entries)/integral
            entries = entries/integral
        else:
            entries, edges = np.histogram(
                data,
                range = (var[1][1], var[1][2]),
                bins = bins       
            )
            yerr = np.sqrt(entries)
            
        bin_width = edges[1]-edges[0]
        bin_centers = edges[:-1] + bin_width/2
        ax.errorbar(bin_centers, entries, yerr=yerr, color='r', ls='', alpha=0.6, label='err', elinewidth=0.6)
        
        
    #deal with automatic y-axis scaling funk
    # if normed is True:
    #     #axes = plt.gca()
    #     ax.set_ylim([0,max(entries/integral)*1.1])

    ax.set_xlabel(var[2])
    ax.set_xlim(var[1][1], var[1][2])

    
    return #(entries, edges, yerr)

## Exploring distributions in MC

   * Photon combinatorials
   * Events Generated using GenePi
      * Currently using non-OSG recon
      * *Should* be fine, but running events through OSG
  

In [5]:
bins = 200
distros = [
    ["Pi0IM_g1g2", (bins,0,0.2), r"$M_{\gamma\gamma}$ $(GeV/c^{2})$"],
    ["Pi0MM2_total", (bins,-5, 5), r"$MM^{2}_{e'n'\gamma\gamma X}$   ($GeV^{2}/c^{4}$)"],
    ["Pi0MM_rec_recoil", (bins,0, 2), r"$MM_{e'\gamma\gamma X}$   ($GeV/c^{2}$)"],
    ["Pi0MM_rec_spectator", (bins,0, 2), r"$MM_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],    
    ["Pi0MP_rec_spectator", (bins,0, 1), r"$MP_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],    
    ["Pi0MP_total", (bins,0, 3), r"$MP_{e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],
    ["Pi0MPt_total", (bins,0, 3), r"$MP^{T}_{e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],
    ["Pi0cop_Nvg_Nnew", (bins,-50, 50), r"$d\phi_{n\gamma^{*}-n\pi_{0}}$ ($\circ$)"],
    ["Pi0pi0coneangle", (bins,0, 60), r"$\theta^{cone}_{\pi_{0}X}$ ($\circ$)"],
    ["Pi0recoilconeangle", (bins,0, 60), r"$\theta^{cone}_{nX}$ ($\circ$)"],
    ["Pi0eg1coneangle", (bins,0, 60), r"$\theta^{cone}_{e\gamma_{1}}$ ($\circ$)"],
    ["Pi0eg2coneangle", (bins,0, 60), r"$\theta^{cone}_{e\gamma_{2}}$ ($\circ$)"]
]

truth_distros = [
    ["truPi0IM_g1g2", (bins,0,0.2), r"$M_{\gamma\gamma}$ $(GeV/c^{2})$"],
    ["truPi0MM2_total", (bins,-5, 5), r"$MM^{2}_{e'n'\gamma\gamma X}$   ($GeV^{2}/c^{4}$)"],
    ["truPi0MM_rec_recoil", (bins,0, 2), r"$MM_{e'\gamma\gamma X}$   ($GeV/c^{2}$)"],
    ["truPi0MM_rec_spectator", (bins,0, 2), r"$MM_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],    
    ["truPi0MP_rec_spectator", (bins,0, 1), r"$MP_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],  
    ["truPi0MP_total", (bins,0, 3), r"$MP_{e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],
    ["truPi0MPt_total", (bins,0, 3), r"$MP^{T}_{e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],
    ["truPi0cop_Nvg_Nnew", (bins,-50, 50), r"$d\phi_{n\gamma^{*}-n\pi_{0}}$ ($\circ$)"],
    ["truPi0pi0coneangle", (bins,0, 60), r"$\theta^{cone}_{\pi_{0}X}$ ($\circ$)"],
    ["truPi0recoilconeangle", (bins,0, 60), r"$\theta^{cone}_{nX}$ ($\circ$)"],
    ["truPi0eg1coneangle", (bins,0, 60), r"$\theta^{cone}_{e\gamma_{1}}$ ($\circ$)"],
    ["truPi0eg2coneangle", (bins,0, 60), r"$\theta^{cone}_{e\gamma_{2}}$ ($\circ$)"]
]

## Raw MC, "Truth-Matched" and data

In [6]:
fig, ax = plt.subplots(12,3, figsize=(12,25))

for i, var in enumerate(distros):
    hist1D(ax[i][0], MC_df, var)
    
for i, var in enumerate(distros):
    hist1D(ax[i][1], MC_df, var, cuts=(MC_df.Truth==1))

# for i, var in enumerate(distros):
#     hist1D(ax[i][2], data_df, var)
    
for i, var in enumerate(truth_distros):
    hist1D(ax[i][2], MC_df, var, cuts=(MC_df.Truth==1))
    
ax[0][0].set_title("MC", y=1)
ax[0][1].set_title("truth-matched", y=1)
ax[0][2].set_title("truth", y=1)

plt.tight_layout(pad=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Cleaning up with some cuts: 
   * 3$\sigma$ $\pi_{0}$-mass cut
   * Neutrons
      * CND only
      * |p| > 350 MeV
   * Photons
      * Require hit in the PCAL

In [7]:
cuts = [
    (MC_df.Pi0flag_cut_3sigPi0IM == 1),
    ((MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)),
    (MC_df.Pi0rec_magP >= 0.35),
    (MC_df.hitPCAL == 1)
]

cuts_truth = [
    (MC_df.Pi0flag_cut_3sigPi0IM == 1),
    ((MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)),
    (MC_df.Pi0rec_magP >= 0.35),
    (MC_df.hitPCAL == 1),
    (MC_df.Truth==1)
]

# dcuts = [
#     (data_df.Pi0flag_cut_3sigPi0IM == 1),
#     ((data_df.Pi0rec_status > 3990) & (data_df.Pi0rec_status < 4150)),
#     (data_df.Pi0rec_magP >= 0.35),
#     (data_df.hitPCAL == 1)
# ]

# oneT_cuts = [
#     (MC_1T_df.Pi0flag_cut_3sigPi0IM == 1),
#     ((MC_1T_df.Pi0rec_status > 3990) & (MC_1T_df.Pi0rec_status < 4150)),
#     (MC_1T_df.Pi0rec_magP >= 0.35),
#     (MC_1T_df.hitPCAL == 1)
# ]
# # cuts_truth = [
#     cuts,
#     (MC_df.Truth==1)
# ]

cut = ft.reduce(lambda x, y: x & y, cuts[:])
t_cut = ft.reduce(lambda x, y: x & y, cuts_truth[:])
#dcut = ft.reduce(lambda x, y: x & y, dcuts[:])

#oT_cut = ft.reduce(lambda x, y: x & y, oneT_cuts[:])

In [8]:
tcut = cut & (MC_df.Truth==1)


In [9]:
(t_cut==tcut).all()

True

In [16]:
fig, ax = plt.subplots(12,3, figsize=(12,25))

for i, var in enumerate(distros):
    hist1D(ax[i][0], MC_df, var, cuts=cut)
    hist1D(ax[i][1], MC_df, var, cuts=t_cut)
    #hist1D(ax[i][2], data_df, var, cuts=dcut)

for i, var in enumerate(truth_distros):
    hist1D(ax[i][2], MC_df, var, cuts=t_cut)


# for i, var in enumerate(truth_distros):
#     hist1D(ax[i][1], MC_df, var, cuts=cut)

ax[0][0].set_title("Reconstructed", y=1)
ax[0][1].set_title("Truth-Matched", y=1)
ax[0][2].set_title("Generated", y=1)

    
plt.tight_layout(pad=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Drift and Broadening of true values

In [17]:
fig1, ax1 = plt.subplots(4,2,figsize=(12,15))
                         
var = [
["Pi0MP_rec_spectator", (bins,0, 1), r"$MP_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],
["truPi0MP_rec_spectator", (bins,0, 1), r"TRUTH $MP_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"]
]

hist1D(ax1[0][0], MC_df, var[0], cuts=t_cut)
hist1D(ax1[0][1], MC_df, var[1], cuts=t_cut)
hist1D(ax1[1][0], MC_df, var[0], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.7)))
hist1D(ax1[1][1], MC_df, var[1], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.7)))
hist1D(ax1[2][0], MC_df, var[0], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.4)))
hist1D(ax1[2][1], MC_df, var[1], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.4)))
hist1D(ax1[3][0], MC_df, var[0], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.2)))
hist1D(ax1[3][1], MC_df, var[1], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.2)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fig1, ax1 = plt.subplots(1,2,figsize=(12,5))
                         
var = [
["Pi0MP_rec_spectator", (bins,0, 1), r"$MP_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],
["truPi0MP_rec_spectator", (bins,0, 1), r"TRUTH $MP_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"]
]

hist1D(ax1[0], MC_df, var[0], cuts=t_cut)
hist1D(ax1[1], MC_df, var[1], cuts=t_cut)
hist1D(ax1[0], MC_df, var[0], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.7)))
hist1D(ax1[1], MC_df, var[1], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.7)))
hist1D(ax1[0], MC_df, var[0], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.4)))
hist1D(ax1[1], MC_df, var[1], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.4)))
hist1D(ax1[0], MC_df, var[0], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.2)))
hist1D(ax1[1], MC_df, var[1], cuts=(t_cut & (MC_df.Pi0MP_rec_spectator < 0.2)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Kinetic energy of the neutron as a potential descriminatory variable

In [13]:
T_distros = [
    ["Pi0recoil_T", (bins,0,2), r"$T_{neutron}$ $(GeV)$"],
    ["Pi0recon_recoil_T", (bins,0,10), r"$recon. T_{neutron}$ $(GeV)$"],
    ["Pi0dneutT", (bins,0,10), r"$\delta T_{neutron}$ $(GeV)$"]
]

truth_T_distros = [
    ["truPi0recoil_T", (bins,0,2), r"$T_{neutron}$ $(GeV)$"],
    ["truPi0recon_recoil_T", (bins,0,10), r"$recon. T_{neutron}$ $(GeV)$"],
    ["truPi0dneutT", (bins,0,10), r"$\delta T_{neutron}$ $(GeV)$"]
]

In [14]:
fig, ax = plt.subplots(3,3, figsize=(10,9))

for i, var in enumerate(T_distros):
    hist1D(ax[i][0], MC_df, var)
    
for i, var in enumerate(T_distros):
    hist1D(ax[i][1], MC_df, var, cuts=(MC_df.Truth==1))

for i, var in enumerate(truth_T_distros):
    hist1D(ax[i][2], MC_df, var, cuts=(MC_df.Truth==1))
    
ax[0][0].set_title("MC", y=1)
ax[0][1].set_title("truth-matched", y=1)
ax[0][2].set_title("truth", y=1)

    
plt.tight_layout(pad=0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fig, ax = plt.subplots(3,3, figsize=(10,9))

for i, var in enumerate(T_distros):
    hist1D(ax[i][0], MC_df, var, cuts=cut)
    hist1D(ax[i][1], MC_df, var, cuts=t_cut)

for i, var in enumerate(truth_T_distros):    
    hist1D(ax[i][2], MC_df, var, cuts=t_cut)


# for i, var in enumerate(truth_T_distros):
#     hist1D(ax[i][1], MC_df, var, cuts=cut)

ax[0][0].set_title("MC", y=1)
ax[0][1].set_title("truth-matched", y=1)
ax[0][2].set_title("truth", y=1)

    
plt.tight_layout(pad=0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig, ax = plt.subplots(12,2, figsize=(12,30))

raw_para = {
    'label': "Raw"
}
traw_para = {
    'label': "Truth-matched Raw"
}
cut_para = {
    'label': "Cut"
}
tcut_para = {
    'label': "Truth-matched Cut"
}

for i, var in enumerate(distros):
    hist1D(ax[i][0], MC_df, var, density=True, label='raw')
    hist1D(ax[i][0], MC_df, var, cuts=(MC_df.Truth==1), density=True, label="Truth-matched Raw")



for i, var in enumerate(distros):
    hist1D(ax[i][1], MC_df, var, cuts=cut, density=True, label="Cut")
    hist1D(ax[i][1], MC_df, var, cuts=t_cut, density=True, label="Truth-matched Cut")
    
for l in 0, 2, 4, 6, 8, 10:
    ax[l][0].legend()
    ax[l][1].legend()
    
plt.tight_layout(pad=0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
list(MC_df.columns)

['Pi0helicity',
 'Pi0W2',
 'Pi0Q2',
 'Pi0xB',
 'Pi0tneg',
 'Pi0tneg_pi0',
 'Pi0dtneg',
 'Pi0IM_g1g2',
 'Pi0MM2_total',
 'Pi0MP_total',
 'Pi0ME_total',
 'Pi0MPt_total',
 'Pi0pi0coneangle',
 'Pi0eg1coneangle',
 'Pi0eg2coneangle',
 'Pi0recoilconeangle',
 'Pi0MP_rec_recoil',
 'Pi0MPt_rec_recoil',
 'Pi0MM_rec_recoil',
 'Pi0MM2_rec_recoil',
 'Pi0MP_rec_spectator',
 'Pi0MPt_rec_spectator',
 'Pi0MM_rec_spectator',
 'Pi0MM2_rec_spectator',
 'Pi0phi_Nvg',
 'Pi0phi_Nnew',
 'Pi0phi_vgnew',
 'Pi0cop_Nvg_vgnew',
 'Pi0cop_Nvg_Nnew',
 'Pi0cop_Nnew_vgnew',
 'Pi0recon_recoil_T',
 'Pi0recoil_T',
 'Pi0dneutT',
 'Pi0rec_px',
 'Pi0rec_py',
 'Pi0rec_pz',
 'Pi0rec_E',
 'Pi0rec_magP',
 'Pi0rec_pT',
 'Pi0rec_theta',
 'Pi0rec_phi',
 'Pi0rec_status',
 'Pi0rec_PID',
 'Pi0e_px',
 'Pi0e_py',
 'Pi0e_pz',
 'Pi0e_E',
 'Pi0e_magP',
 'Pi0e_pT',
 'Pi0e_theta',
 'Pi0e_phi',
 'Pi0e_status',
 'Pi0phot1_px',
 'Pi0phot1_py',
 'Pi0phot1_pz',
 'Pi0phot1_E',
 'Pi0phot1_magP',
 'Pi0phot1_pT',
 'Pi0phot1_theta',
 'Pi0phot1_phi',
 '

In [22]:
bins = 200
evo_distros1 = [
    ["Pi0IM_g1g2", (bins,0,0.2), r"$M_{\gamma\gamma}$ $(GeV/c^{2})$"],
    ["Pi0MM2_total", (bins,-5, 5), r"$MM^{2}_{e'n'\gamma\gamma X}$   ($GeV^{2}/c^{4}$)"]
]

evo_distros2 = [
    ["Pi0MM_rec_recoil", (bins,0, 2), r"$MM_{e'\gamma\gamma X}$   ($GeV/c^{2}$)"],
    ["Pi0MM_rec_spectator", (bins,0, 2), r"$MM_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],    
    ["Pi0MP_rec_spectator", (bins,0, 0.5), r"$MP_{eD -> e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],    
    ["Pi0MP_total", (bins,0, 3), r"$MP_{e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"],
    ["Pi0MPt_total", (bins,0, 3), r"$MP^{T}_{e'n'\gamma\gamma X}$   ($GeV/c^{2}$)"]
]

evo_distros3 = [
    ["Pi0cop_Nvg_Nnew", (bins,-50, 50), r"$d\phi_{n\gamma^{*}-n\pi_{0}}$ ($\circ$)"],
    ["Pi0pi0coneangle", (bins,0, 60), r"$\theta^{cone}_{\pi_{0}X}$ ($\circ$)"],
    ["Pi0recoilconeangle", (bins,0, 60), r"$\theta^{cone}_{nX}$ ($\circ$)"],
    ["Pi0eg1coneangle", (bins,0, 60), r"$\theta^{cone}_{e\gamma_{1}}$ ($\circ$)"],
    ["Pi0eg2coneangle", (bins,0, 60), r"$\theta^{cone}_{e\gamma_{2}}$ ($\circ$)"]
]


In [23]:
cuts_truth = [
    (MC_df.Truth==1),
    (MC_df.Pi0flag_cut_3sigPi0IM == 1),
    (MC_df.Pi0rec_magP >= 0.35),
    ((MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)),
    (MC_df.hitPCAL == 1)
]

cut_label = [
    "Truth-Match",
    r"$3\sigma$  $\pi_{0}$-mass",
    r"$|p|_{neut.} > 350MeV$",    
    "CND neutron",
    "enforce PCAL hit"
]

In [24]:
fig1, ax1 = plt.subplots(5,2,figsize=(7,12))

for j in range(len(cuts_truth)):
    for i, var in enumerate(evo_distros1):
        
        hist1D(ax1[j][i], MC_df, var, cuts=(ft.reduce(lambda x, y: x & y, cuts_truth[:j+1])))

for k in range(len(cuts_truth)):
    ax1[k][0].set_ylabel(cut_label[k])

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig1, ax1 = plt.subplots(5,5,figsize=(17,12))

for j in range(len(cuts_truth)):
    for i, var in enumerate(evo_distros2):
        
        hist1D(ax1[j][i], MC_df, var, cuts=(ft.reduce(lambda x, y: x & y, cuts_truth[:j+1])))

for k in range(len(cuts_truth)):
    ax1[k][0].set_ylabel(cut_label[k])

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
fig1, ax1 = plt.subplots(5,5,figsize=(17,12))

for j in range(len(cuts_truth)):
    for i, var in enumerate(evo_distros3):
        
        hist1D(ax1[j][i], MC_df, var, cuts=(ft.reduce(lambda x, y: x & y, cuts_truth[:j+1])))

for k in range(len(cuts_truth)):
    ax1[k][0].set_ylabel(cut_label[k])

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Add PID to truth

In [27]:
fig, ax = plt.subplots(12,3, figsize=(12,25))

for i, var in enumerate(distros):
    hist1D(ax[i][0], MC_df, var, cuts=cut, density=True, label="MC cut", color="royalblue")
    #hist1D(ax[i][0], MC_df, var, cuts=cut&(MC_df.Pi0flag_MC_neutrec==1), bins=75, density=True, label="truth+PID")

    hist1D(ax[i][1], MC_df, var, cuts=t_cut, density=True, label="truth-matched")
    hist1D(ax[i][1], MC_df, var, cuts=t_cut&(MC_df.Pi0flag_MC_neutrec==1), bins=75, density=True, label="truth+PID")

    #hist1D(ax[i][2], data_df, var, cuts=dcut&(MC_df.Pi0flag_MC_neutrec==1))

for i, var in enumerate(truth_distros):
    hist1D(ax[i][2], MC_df, var, cuts=t_cut, density=True, label="truth-matched")
    hist1D(ax[i][2], MC_df, var, cuts=t_cut&(MC_df.Pi0flag_MC_neutrec==1), bins=75, density=True, label="truth+PID")



# for i, var in enumerate(truth_distros):
#     hist1D(ax[i][1], MC_df, var, cuts=cut&(MC_df.Pi0flag_MC_neutrec==1))

ax[0][0].set_title("MC", y=1)
ax[0][1].set_title("truth-matched", y=1)
ax[0][2].set_title("truth", y=1)

# for i in 1, 4, 7, 10:
#     ax[i][2].legend()
#     ax[i][1].legend()
#     ax[i][0].legend()
    
ax[0][1].legend(loc=2)
ax[3][1].legend(loc=2)
ax[6][1].legend(loc=1)
ax[9][1].legend(loc=1)


plt.tight_layout(pad=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
fig, ax = plt.subplots(12,3, figsize=(12,25))

for i, var in enumerate(distros):
    hist1D(ax[i][0], MC_df, var, cuts=cut, density=True, label="MC cut", color="royalblue")
    #hist1D(ax[i][0], MC_df, var, cuts=cut&(MC_df.Pi0flag_MC_neutrec==1), bins=75, label="truth+PID")

    hist1D(ax[i][1], MC_df, var, cuts=t_cut, label="truth-matched")
    hist1D(ax[i][1], MC_df, var, cuts=t_cut&(MC_df.Pi0flag_MC_neutrec==1), bins=75, label="truth+PID")

    #hist1D(ax[i][2], data_df, var, cuts=dcut&(MC_df.Pi0flag_MC_neutrec==1))

for i, var in enumerate(truth_distros):
    hist1D(ax[i][2], MC_df, var, cuts=t_cut, label="truth-matched")
    hist1D(ax[i][2], MC_df, var, cuts=t_cut&(MC_df.Pi0flag_MC_neutrec==1), bins=75, label="truth+PID")



# for i, var in enumerate(truth_distros):
#     hist1D(ax[i][1], MC_df, var, cuts=cut&(MC_df.Pi0flag_MC_neutrec==1))

ax[0][0].set_title("MC", y=1)
ax[0][1].set_title("truth-matched", y=1)
ax[0][2].set_title("truth", y=1)

for i in 1, 4, 7, 10:
    ax[i][2].legend()
    ax[i][1].legend()
    ax[i][0].legend()
    
plt.tight_layout(pad=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
fig, ax = plt.subplots(3,3, figsize=(10,9))

for i, var in enumerate(T_distros):
    hist1D(ax[i][0], MC_df, var, cuts=cut, density=True)
    hist1D(ax[i][1], MC_df, var, cuts=t_cut, density=True)
    
    hist1D(ax[i][1], MC_df, var, cuts=t_cut&(MC_df.Pi0flag_MC_neutrec==1), density=True, bins=75)

for i, var in enumerate(truth_T_distros):    
    hist1D(ax[i][2], MC_df, var, cuts=t_cut, density=True)
    hist1D(ax[i][2], MC_df, var, cuts=t_cut&(MC_df.Pi0flag_MC_neutrec==1), density=True, bins=75)


# for i, var in enumerate(truth_T_distros):
#     hist1D(ax[i][1], MC_df, var, cuts=cut, density=True)

ax[0][0].set_title("MC", y=1)
ax[0][1].set_title("truth-matched", y=1)
ax[0][2].set_title("truth", y=1)

    
plt.tight_layout(pad=0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …